In [1]:
import toml
import pandas as pd
import re
import os
import random

# Add the scripts directory to the system path
import sys
sys.path.insert(0, '../scripts/')

# Now import the class
from data_preprocessing import DataPreprocessor


project_root = "../"
os.chdir(project_root)

print("Current Working Directory: ", os.getcwd())


Current Working Directory:  /home/b/code/nlu-engine


In [2]:
preprocessor = DataPreprocessor(config_path="config/config.toml")

In [3]:
# Load the unprocessed dataset
df = pd.read_csv(preprocessor.config["data_file"], sep=',')

In [4]:
df

,utterance,domain,intent,annotated_utterance,entity_types,annotated_utterance_cleaned,annotated_utterance_cleaned_and_removed_incorrect_tags
0,wake me up at 9am,alarm,alarm_set,wake me up at [time : 9am],time,wake me up at [time : 9am],wake me up at [time : 9am]
1,would you further rephrase me about your words.,general,explain,would you further rephrase me about your words.,NaN,would you further rephrase me about your words.,would you further rephrase me about your words.
2,"olly that's all, stop it.",general,commandstop,"olly that's all, stop it.",NaN,"olly that's all, stop it.","olly that's all, stop it."
3,what is on my to do list today,lists,lists_query,what is on my [list_name : to do list] today,list_name,what is on my [list_name : to do list] today,what is on my [list_name : to do list] today
4,book tomorrow's train ticket for me to bombay ...,transport,ticket,book [time : tomorrow's] [transport_type : tra...,"time, transport_type, place_name",book [time : tomorrow's] [transport_type : tra...,book [time : tomorrow's] [transport_type : tra...
...,...,...,...,...,...,...,...
339,can you shut up please.,audio,volume_mute,can you shut up please.,NaN,can you shut up please.,can you shut up please.
340,read me the content of this list,lists,lists_query,read me the content of this list,NaN,read me the content of this list,read me the content of this list
341,whats my schedule like,calendar,calendar_query,whats my schedule like,NaN,whats my schedule like,whats my schedule like
342,give me music news,music,music_query,give me [media_type : music] news,NaN,give me [media_type : music] news,give me music news


In [6]:
# Clean and structure data
df = preprocessor.clean_and_structure_data()
df

,utterance,domain,intent,annotated_utterance
0,wake me up at 9am,alarm,alarm_set,wake me up at [time : 9am]
1,would you further rephrase me about your words.,general,explain,would you further rephrase me about your words.
2,"olly that's all, stop it.",general,commandstop,"olly that's all, stop it."
3,what is on my to do list today,lists,lists_query,what is on my [list_name : to do list] today
4,book tomorrow's train ticket for me to bombay ...,transport,ticket,book [time : tomorrow's] [transport_type : tra...
...,...,...,...,...
339,can you shut up please.,audio,volume_mute,can you shut up please.
340,read me the content of this list,lists,lists_query,read me the content of this list
341,whats my schedule like,calendar,calendar_query,whats my schedule like
342,give me music news,music,music_query,give me music news


In [8]:
# Load CSV into df nlu_data_intent_and_entity_refined_df.csv
nlu_data_intent_and_entity_refined_df = pd.read_csv('data/nlu_data_intent_and_entity_refined_df.csv', sep=',')
nlu_data_intent_and_entity_refined_df

,Unnamed: 0,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation
0,0,1.0,1.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 5am this week,wake me up at five am this week,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : five am] [date : this week]
1,1,1.0,2.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 9am on Friday,wake me up at nine am on friday,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : nine am] on [date : friday]
2,2,1.0,3.0,NaN,Write what you would tell your PDA in the foll...,"date, time",set an alarm for two hours from now,set an alarm for two hours from now,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,set an alarm for [time : two hours from now]
3,3,1.0,31.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly quiet!,quiet,audio,volume_mute,volume_mute,NaN,NaN,NaN,NaN,quiet
4,5,1.0,33.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly pause for ten seconds,pause for ten seconds,audio,volume_mute,volume_mute,True,NaN,True,NaN,pause for [time : ten seconds]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25069,25711,NaN,781.0,NaN,NaN,NaN,"that's cool, musch appreciated, olly.","that's cool, musch appreciated, olly.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, musch appreciated, olly."
25070,25712,NaN,782.0,NaN,NaN,NaN,"you are hero, appreciated.","you are hero, appreciated.",general,praise,praise,NaN,NaN,NaN,NaN,"you are hero, appreciated."
25071,25713,NaN,783.0,NaN,NaN,NaN,"thanks, that's nice.","thanks, that's nice.",general,praise,praise,NaN,NaN,NaN,NaN,"thanks, that's nice."
25072,25714,NaN,784.0,NaN,NaN,NaN,"that's cool, thank you so much.","that's cool, thank you so much.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, thank you so much."


In [9]:
# Remove enteries where df['utterance'] is in nlu_data_intent_and_entity_refined_df['utterance']
nlu_data_intent_and_entity_refined_df = nlu_data_intent_and_entity_refined_df[~nlu_data_intent_and_entity_refined_df['answer_normalised'].isin(df['utterance'])]

In [10]:
# Get the number of unique intents
nlu_data_intent_and_entity_refined_df['intent'].nunique()

67

In [11]:
# Remove all entries with remove == True
nlu_data_intent_and_entity_refined_df = nlu_data_intent_and_entity_refined_df[nlu_data_intent_and_entity_refined_df['remove'] != True]
nlu_data_intent_and_entity_refined_df

,Unnamed: 0,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation
0,0,1.0,1.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 5am this week,wake me up at five am this week,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : five am] [date : this week]
1,1,1.0,2.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 9am on Friday,wake me up at nine am on friday,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : nine am] on [date : friday]
2,2,1.0,3.0,NaN,Write what you would tell your PDA in the foll...,"date, time",set an alarm for two hours from now,set an alarm for two hours from now,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,set an alarm for [time : two hours from now]
3,3,1.0,31.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly quiet!,quiet,audio,volume_mute,volume_mute,NaN,NaN,NaN,NaN,quiet
6,7,1.0,38.0,NaN,How would you ask your PDA to change the light...,color_type,make the lighting bit more warm here,make the lighting a bit more warm here,iot,hue_lightchange,hue_lightup,True,NaN,False,NaN,make the lighting a bit more [color_type : war...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25069,25711,NaN,781.0,NaN,NaN,NaN,"that's cool, musch appreciated, olly.","that's cool, musch appreciated, olly.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, musch appreciated, olly."
25070,25712,NaN,782.0,NaN,NaN,NaN,"you are hero, appreciated.","you are hero, appreciated.",general,praise,praise,NaN,NaN,NaN,NaN,"you are hero, appreciated."
25071,25713,NaN,783.0,NaN,NaN,NaN,"thanks, that's nice.","thanks, that's nice.",general,praise,praise,NaN,NaN,NaN,NaN,"thanks, that's nice."
25072,25714,NaN,784.0,NaN,NaN,NaN,"that's cool, thank you so much.","that's cool, thank you so much.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, thank you so much."


In [12]:
# Remove all entries where intent and predicted_label are not the same
nlu_data_intent_and_entity_refined_df = nlu_data_intent_and_entity_refined_df[nlu_data_intent_and_entity_refined_df['intent'] == nlu_data_intent_and_entity_refined_df['predicted_label']]
nlu_data_intent_and_entity_refined_df

,Unnamed: 0,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation
0,0,1.0,1.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 5am this week,wake me up at five am this week,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : five am] [date : this week]
1,1,1.0,2.0,NaN,Write what you would tell your PDA in the foll...,"date, time",wake me up at 9am on Friday,wake me up at nine am on friday,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,wake me up at [time : nine am] on [date : friday]
2,2,1.0,3.0,NaN,Write what you would tell your PDA in the foll...,"date, time",set an alarm for two hours from now,set an alarm for two hours from now,alarm,alarm_set,alarm_set,NaN,NaN,NaN,NaN,set an alarm for [time : two hours from now]
3,3,1.0,31.0,NaN,Write what you would tell your PDA in the foll...,player_setting,Olly quiet!,quiet,audio,volume_mute,volume_mute,NaN,NaN,NaN,NaN,quiet
8,9,1.0,40.0,NaN,How would you ask your PDA to turn the lights ...,NaN,turn the lights off please,turn the lights off please,iot,hue_lightoff,hue_lightoff,NaN,NaN,NaN,NaN,turn the lights off please
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25069,25711,NaN,781.0,NaN,NaN,NaN,"that's cool, musch appreciated, olly.","that's cool, musch appreciated, olly.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, musch appreciated, olly."
25070,25712,NaN,782.0,NaN,NaN,NaN,"you are hero, appreciated.","you are hero, appreciated.",general,praise,praise,NaN,NaN,NaN,NaN,"you are hero, appreciated."
25071,25713,NaN,783.0,NaN,NaN,NaN,"thanks, that's nice.","thanks, that's nice.",general,praise,praise,NaN,NaN,NaN,NaN,"thanks, that's nice."
25072,25714,NaN,784.0,NaN,NaN,NaN,"that's cool, thank you so much.","that's cool, thank you so much.",general,praise,praise,NaN,NaN,NaN,NaN,"that's cool, thank you so much."


In [13]:
# For each intent randomly sample 10 entries
nlu_data_intent_and_entity_refined_df = nlu_data_intent_and_entity_refined_df.groupby('intent').apply(lambda x: x.sample(n=10)).reset_index(drop=True)
nlu_data_intent_and_entity_refined_df

,Unnamed: 0,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation
0,20677,260.0,23585.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add a new email in my contacts from John,add a new email in my contacts from john,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add a new email in my contacts from [person : ...
1,19356,142.0,4312.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add work email address,add work email address,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add work email address
2,19474,219.0,6114.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add this email to my address book.,add this email to my address book,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add this email to my address book
3,20685,920.0,23671.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add this email to my contact,add this email to my contact,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add this email to my contact
4,19413,182.0,5091.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add jpearsonjessica@gmail.com to email contacts,add jpearsonjessica@gmail dot com to email con...,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add [email_address : jpearsonjessica@gmail dot...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,4682,615.0,15976.0,NaN,How would you ask your PDA to turn off or on a...,socket_type,power on wemo,power on wemo,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,power on [device_type : wemo]
606,6582,935.0,24023.0,NaN,Write what you would tell your PDA in the foll...,socket_type,flip the wemo on,flip the wemo on,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,flip the [device_type : wemo] on
607,3335,461.0,11751.0,NaN,Write what you would tell your PDA in the foll...,socket_type,turn on plug,turn on plug,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,turn on plug
608,702,68.0,2981.0,NaN,Write what you would tell your PDA in the foll...,socket_type,turn on the tv socket when I arrive home,turn on the tv socket when i arrive home,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,turn on the [device_type : tv] socket when i a...


In [14]:
# Drop duplicates
nlu_data_intent_and_entity_refined_df = nlu_data_intent_and_entity_refined_df.drop_duplicates()
nlu_data_intent_and_entity_refined_df

,Unnamed: 0,userid,answerid,notes,question,suggested_entities,answer,answer_normalised,scenario,intent,predicted_label,intent_refined,entity_refined,remove,status,answer_annotation
0,20677,260.0,23585.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add a new email in my contacts from John,add a new email in my contacts from john,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add a new email in my contacts from [person : ...
1,19356,142.0,4312.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add work email address,add work email address,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add work email address
2,19474,219.0,6114.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add this email to my address book.,add this email to my address book,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add this email to my address book
3,20685,920.0,23671.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add this email to my contact,add this email to my contact,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add this email to my contact
4,19413,182.0,5091.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add jpearsonjessica@gmail.com to email contacts,add jpearsonjessica@gmail dot com to email con...,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add [email_address : jpearsonjessica@gmail dot...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,4682,615.0,15976.0,NaN,How would you ask your PDA to turn off or on a...,socket_type,power on wemo,power on wemo,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,power on [device_type : wemo]
606,6582,935.0,24023.0,NaN,Write what you would tell your PDA in the foll...,socket_type,flip the wemo on,flip the wemo on,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,flip the [device_type : wemo] on
607,3335,461.0,11751.0,NaN,Write what you would tell your PDA in the foll...,socket_type,turn on plug,turn on plug,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,turn on plug
608,702,68.0,2981.0,NaN,Write what you would tell your PDA in the foll...,socket_type,turn on the tv socket when I arrive home,turn on the tv socket when i arrive home,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,turn on the [device_type : tv] socket when i a...


In [15]:
# change the following column names: answer_normalised -> utterance, scenario -> domain, answer_annotation -> annotated_utterance
nlu_data_intent_and_entity_refined_df = nlu_data_intent_and_entity_refined_df.rename(columns={'answer_normalised': 'utterance', 'scenario': 'domain', 'answer_annotation': 'annotated_utterance'})
nlu_data_intent_and_entity_refined_df


,Unnamed: 0,userid,answerid,notes,question,suggested_entities,answer,utterance,domain,intent,predicted_label,intent_refined,entity_refined,remove,status,annotated_utterance
0,20677,260.0,23585.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add a new email in my contacts from John,add a new email in my contacts from john,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add a new email in my contacts from [person : ...
1,19356,142.0,4312.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add work email address,add work email address,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add work email address
2,19474,219.0,6114.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add this email to my address book.,add this email to my address book,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add this email to my address book
3,20685,920.0,23671.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add this email to my contact,add this email to my contact,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add this email to my contact
4,19413,182.0,5091.0,NaN,Write what you would tell your PDA in the foll...,"email_address, mail_content, mail_title",Add jpearsonjessica@gmail.com to email contacts,add jpearsonjessica@gmail dot com to email con...,email,addcontact,addcontact,NaN,NaN,NaN,NaN,add [email_address : jpearsonjessica@gmail dot...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,4682,615.0,15976.0,NaN,How would you ask your PDA to turn off or on a...,socket_type,power on wemo,power on wemo,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,power on [device_type : wemo]
606,6582,935.0,24023.0,NaN,Write what you would tell your PDA in the foll...,socket_type,flip the wemo on,flip the wemo on,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,flip the [device_type : wemo] on
607,3335,461.0,11751.0,NaN,Write what you would tell your PDA in the foll...,socket_type,turn on plug,turn on plug,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,turn on plug
608,702,68.0,2981.0,NaN,Write what you would tell your PDA in the foll...,socket_type,turn on the tv socket when I arrive home,turn on the tv socket when i arrive home,iot,wemo_on,wemo_on,NaN,NaN,NaN,NaN,turn on the [device_type : tv] socket when i a...


In [16]:
# Keep the following columns: utterance, domain, intent, annotated_utterance
sampled_nlu_df = nlu_data_intent_and_entity_refined_df[['utterance', 'domain', 'intent', 'annotated_utterance']]
sampled_nlu_df

,utterance,domain,intent,annotated_utterance
0,add a new email in my contacts from john,email,addcontact,add a new email in my contacts from [person : ...
1,add work email address,email,addcontact,add work email address
2,add this email to my address book,email,addcontact,add this email to my address book
3,add this email to my contact,email,addcontact,add this email to my contact
4,add jpearsonjessica@gmail dot com to email con...,email,addcontact,add [email_address : jpearsonjessica@gmail dot...
...,...,...,...,...
605,power on wemo,iot,wemo_on,power on [device_type : wemo]
606,flip the wemo on,iot,wemo_on,flip the [device_type : wemo] on
607,turn on plug,iot,wemo_on,turn on plug
608,turn on the tv socket when i arrive home,iot,wemo_on,turn on the [device_type : tv] socket when i a...


In [17]:
# export to csv
sampled_nlu_df.to_csv('data/sample_nlu_data.csv', index=False)

In [ ]:
# TODO: rename notebook!
# NOTE: Make sure to set the random number for number of domains higher than the training data. Perhaps 5 or even more for intents!

#TODO: Refactor this code and rename the notebook so these steps are reproducible!

# TODO: See where the model doesn't agree. Maybe the data is wrong instead of the model?

# TODO: Use data to train a new model with the old data, too.

# TODO: Run this model over Mycroft data and/or Snips data and see how it does.

# TODO: Add in data from other resources and re-train and run a final test. 